in the end, we want for each hour the following:
- DE day-ahead price (+ market closing time)
- CH day-ahead price (+ market closing time)
- JAO Auction price DE->CH (+ market closing time)
- JAO Auction price (CH->DE) (+ market closing time)
- for each market closing time some additional predicted weather data or other potential explaining variables

0. make sure, you have the some day-ahead price data for both countries in data/raw
1. with functions from src/upstream.py we get the data from JAO and put it in data/external
2. create a dataset with day-ahead price difference CH-DE per hour, both auction prices, and further variables for that hour that were known at market closing time.

In [16]:
import sys; sys.path.append("..")

import pandas as pd
import requests
import ast
from datetime import datetime, timedelta
from calendar import monthrange
import os

In [17]:
# Import the fetch_auction_data function
from src.datafeed.upstream import fetch_auction_data

from src.config import KEY

Download the data from JAO (This data is in CET/CEST!)

The dataframe in the first year start on 2th January.

In [19]:
# Define your API key (replace with your actual API key)
api_key = KEY

# Set the start and end years
start_year = 2016
end_year = 2023

corridors = ["de-ch"]

for corridor in corridors:

    # Call the function to fetch and process the data
    auction_data = fetch_auction_data(start_year, end_year, api_key, corridor)

    # Optionally, display the first few rows of the combined DataFrame
    auction_data.head()


/Users/vanshkhanna/Desktop/electricity-price-disparity-ch-de/notebooks/../src/datafeed/upstream.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)


Processing the data.
x until now, all data is in cet/cest. 

To do:
- DE-LU is in quarter-hours. we should weigh the prices with the forecasted loads so we also get hourly data there. 
- make functions in src/datafeed/downstream.py to create a dataframe in data/processed
    - add data for forecasted loads for both regions (this is before the market closes, estimate by the Transmission System Operators)
    - add hourly forecasted weather data (?)
    - add other forecasted data (?)
    - put all in one easily accessible dataframe for the analysis.

In [26]:
# Define a range of years from start_year to end_year (inclusive)
years = range(start_year, end_year + 1)

# Function to load day-ahead price data for a specific bidding zone across multiple years
def load_price_series(bidding_zone, years, path="../data/raw/"):
    dfs = []  # List to store DataFrames for each year
    for year in years:
        # Construct the filename based on the bidding zone and year
        filename = f"{bidding_zone}_Day-ahead Prices_{year}.csv"
        filepath = os.path.join(path, filename)  # Full path to the file
        try:
            # Load the CSV file into a DataFrame
            df = pd.read_csv(filepath)
            print(df.head())
            # Parse the datetime from the "MTU (CET/CEST)" column and use the start time of the interval
            df["MTU (CET/CEST)"] = pd.to_datetime(
                df["MTU (CET/CEST)"].str.split(" - ").str[0],  # Extract the start time of the interval
                format="%d.%m.%Y %H:%M",  # Specify the format of the datetime
                errors="coerce"  # Coerce parsing errors to NaT
            )
            # Drop duplicate timestamps, keeping the first occurrence (means that one hour of each year is missing because CET/CEST changes,
            # but since all dataframes are in CET/CEST, this is not a problem and negligible for our analysis)
            df = df.loc[~df["MTU (CET/CEST)"].duplicated()]
            # Set the datetime column as the index
            df.set_index("MTU (CET/CEST)", inplace=True)
            # Append the "Day-ahead Price" column to the list of DataFrames
            dfs.append(df["Day-ahead Price [EUR/MWh]"])
        except Exception as e:
            # Ignore missing files or other exceptions and continue
            # Uncomment the print statement to debug missing files
            print(f"Missing {filepath}: {e}")
            continue
    # Concatenate all yearly DataFrames if any exist, otherwise return an empty Series
    return pd.concat(dfs) if dfs else pd.Series(dtype=float)

# Load day-ahead price data for each bidding zone
ch_series = load_price_series("CH", years)  # Load data for Switzerland (CH)
de_lu_series = load_price_series("DE-LU", years)  # Load data for Germany-Luxembourg (DE-LU)
de_at_lu_series = load_price_series("DE-AT-LU", years)  # Load data for Germany-Austria-Luxembourg (DE-AT-LU)

# Create a final DataFrame with a unified set of timestamps from all bidding zones
all_timestamps = sorted(set(ch_series.index) | set(de_lu_series.index) | set(de_at_lu_series.index))
df_final = pd.DataFrame(index=all_timestamps)  # Initialize the final DataFrame with all timestamps as the index

# Populate the final DataFrame with price series from each bidding zone
df_final['CH'] = ch_series  # Add Swiss prices
df_final['DE-LU'] = de_lu_series  # Add German-Luxembourg prices
df_final['DE-AT-LU'] = de_at_lu_series  # Add German-Austria-Luxembourg prices

# Loop through each corridor and add auction data to the final DataFrame
for corridor in corridors:
    # Load auction data for the specific corridor
    auction_data = pd.read_csv(f"../data/external/jao_{corridor}.csv")
    # Create a datetime column by combining the date and the start of the productHour range
    auction_data['datetime'] = pd.to_datetime(
        auction_data['date'] + ' ' + auction_data['productHour'].str.split('-').str[0]
    )
    # Drop duplicate rows based on the datetime column
    auction_data = auction_data.drop_duplicates(subset=['datetime'])
    # Set the datetime column as the index
    auction_data = auction_data.set_index('datetime')

    # Add auction-related columns to the final DataFrame
    df_final[f"{corridor}_auctionPrice"] = auction_data[f"{corridor}_auctionPrice"]
    df_final[f"{corridor}_requestedCapacity"] = auction_data[f"{corridor}_requestedCapacity"]
    df_final[f"{corridor}_offeredCapacity"] = auction_data[f"{corridor}_offeredCapacity"]

# Sort the final DataFrame by index (datetime)
df_final.sort_index(inplace=True)

#create folder 
processed_folder = '../data/processed'
os.makedirs(processed_folder, exist_ok=True)

# Save the final merged DataFrame to a CSV file for further analysis
df_final.to_csv('../data/processed/merged_data.csv')

# Display the final DataFrame
df_final

                        MTU (CET/CEST)  Day-ahead Price [EUR/MWh] Currency  \
0  01.01.2016 00:00 - 01.01.2016 01:00                      41.09      EUR   
1  01.01.2016 01:00 - 01.01.2016 02:00                      40.16      EUR   
2  01.01.2016 02:00 - 01.01.2016 03:00                      36.03      EUR   
3  01.01.2016 03:00 - 01.01.2016 04:00                      33.59      EUR   
4  01.01.2016 04:00 - 01.01.2016 05:00                      32.92      EUR   

   BZN|CH  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  
                        MTU (CET/CEST)  Day-ahead Price [EUR/MWh] Currency  \
0  01.01.2017 00:00 - 01.01.2017 01:00                      57.04      EUR   
1  01.01.2017 01:00 - 01.01.2017 02:00                      51.94      EUR   
2  01.01.2017 02:00 - 01.01.2017 03:00                      49.07      EUR   
3  01.01.2017 03:00 - 01.01.2017 04:00                      45.01      EUR   
4  01.01.2017 04:00 - 01.01.2017 05:00                      42.40    

,CH,DE-LU,DE-AT-LU,de-ch_auctionPrice,de-ch_requestedCapacity,de-ch_offeredCapacity
2016-01-01 00:00:00,41.09,NaN,23.86,9.26,1663.0,392.0
2016-01-01 01:00:00,40.16,NaN,22.39,8.50,1678.0,392.0
2016-01-01 02:00:00,36.03,NaN,20.59,8.87,1678.0,392.0
2016-01-01 03:00:00,33.59,NaN,16.81,7.50,1678.0,392.0
2016-01-01 04:00:00,32.92,NaN,17.41,10.02,1668.0,392.0
...,...,...,...,...,...,...
2023-12-31 19:00:00,23.40,9.0,NaN,2.34,3552.0,530.0
2023-12-31 20:00:00,20.65,7.95,NaN,2.34,3701.0,530.0
2023-12-31 21:00:00,9.58,6.0,NaN,3.45,3701.0,530.0
2023-12-31 22:00:00,16.78,10.68,NaN,3.54,3681.0,530.0
